## Podsumowanie poprzednich zajęć 

Na poprzednich dwóch zajęciach laboratoryjnych zrealizowaliśmy zagadnienia dotyczące przetwarzania danych ustrukturyzowanych i nieustrukturyzowanych w trybie wsadowym. Ponadto przygotowaliśmy środowisko produkcyjne (z wykorzystaniem biblioteki FLASK) wykorzystujące model napisany w pełni obiektowo otrzymany po wstępnym przetworzeniu danych (irys).   

1. Ustrukturyzowane dane - tablice numpy, ramki danych Pandas, tabele danych w bazach SQL (tworzenie, filtrowanie, modyfikacja)
2. Nieustrukturyzowane dane - JSON, tensory numpy (odczyt, zapis, przetworzenie)
3. Wykorzystanie obiektowego programowania w Pythonie - podstawa budowy klas, tworzenie obiektów, korzystanie z pól obiektów i metod (funkcji). 
4. Stworzenie modelu klasyfikacji binarnej opartego o sieć perceprtornu oraz wykorzystującą algorytm Adeline napisany obiektowo w pełnej analogii do modeli z biblioteki sklearn.
5. Wykorzystanie środowiska SQLAlchemy do łączenia się z bazami danych. 
6. Strona www realizująca API z wykorzystaniem modelu - nowe dane w czasie rzeczywistym + prognoza - Jako system odpytywania modelu w czasie rzeczywistym (Zastanów się jak go unowocześnić) 


Podczas przerabiania dowolnych technik uczenia maszynowego najczęściej (jeśli nie zawsze) jesteśmy uczeni realizacji zadań takiego systemu z podziałem na trzy podstawowe kategorie:

1. Uczenie nadzorowane  - supervised learning
    - klasyfikacja - zrealizowany na poprzednich ćwiczeniach
    - Regresja liniowa
2. Uczenie nienadzorowane - unsupervised learning
3. Uczenie przez wzmacnianie - reinforcement learning

Jednak systemy te można również klasyfikować ze względu na `możliwość trenowania przyrostowego przy użyciu strumienia nadsyłanych danych`

1. **Uczenie wsadowe - batch learning**. To system w którym do jego nauki musisz wykorzysać wszytkie zapisane i już istniejące dane. Zajmuje zazwyczaj dużo czasu i zasobów - przeprowadzany w trybie offline. System wpierw jest uczony, a następnie zostaje wdrożony do cyklu produkcyjnego i już więcej nie jest trenowany (korzysta tylko ze zdobytych wcześniej informacji). Zajwisko to nazywane jest **uczeniem offline**. 

Jeśli chcesz aby system uczenia wsadowego brał pod uwagę nowe dane to musisz od podstaw wytrenować nową wersję systemu przy użyciu wszystkich dostępnch danych, wyłączyć stary system i zastąpić go nowym. Na szczęście proces ten jest w pełni automatyzowalny. Jednak trzeba pamiętać, iż trenowanie nowego modelu na pełnym zbiorze danych może trwać bardzo długo (i jest dość kosztowne) stąd wymiana modeli pojawia się np raz na tydzień raz na dzień. W przypadku bardzo dużej ilości informacji system taki może szybko przestać działać - zamiast wykonywać swoje zadania będzie obliczał nowy model. 

2. W procesie **uczenia przyrostowego - online learning** system trenowany jest na bieżąco poprzez sekwencyjne dostarczanie danych (pojedyncze lub minipaczki - mini-batches). Każdy krok uczenia jest szybki i mało kosztowny. Uczenie następuje w momencie pojawienia się nowych danych.  

Uczenie przyrostowe sprawdza się wszędzie tam gdzie układ odbiera ciągły strumień danych (urządzenia IoT, giełda) i wymagana jest szybkie i autonomiczne dopasowanie do nowych warunków. Przydaje się również przy pracy z ograniczonymi zasobami obliczeniowymi (stare dane nie są istotne).

Dużym problemem uczenia przyrostowego jest stopniowy spadek wydajności systemu w przypadku gdy dostarczone dane przestają być prawidłowe. Np. uszkodzony czujnik, celowe zasypywanie przeglądarki danymi w celu podbicia rankingu w wynikach wyszukiwania (algorytmy wykrywania anomalii).


[Stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)

[Stochastic learning](https://leon.bottou.org/publications/pdf/mlss-2003.pdf)

## Środowisko Apache SPARK

[książka](https://pages.databricks.com/rs/094-YMS-629/images/LearningSpark2.0.pdf) 


1. Silnik analityczny do przetwarzania danych na dużą skalę
2. Projekt open source, od 2013 w Apache Software FOundation
3. Napisany w Scali 
4. Udostępnia API w Java, Scala, Python, R 


### Instalacja i uruchomienie 

1. Wersja trywialna (Docker) 

```{bash}
docker run -d -p 8888:8888 -v "full_path_to_your_folder:/notebooks" sebkaz/docker-spark-jupyter
```

```{bash}
docker run -d -p 8888:8888 -v "full_path_to_your_folder:/notebooks" jupyter/pyspark-notebook
```

2. Wersja trywialna trywialna (komputer ze środowiskiem Python + JDK JAVA przynajmniej w wersji 8) 

    - [Ściągnij katalog](https://www.apache.org/dyn/closer.lua/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz)
    - Rozpakuj np 7z
    - umieść w wygodnym miejscu i zapisz ścieżkę (będzie potrzebna do findspark() )
    - uruchom jupyter notebook'a


In [1]:
import findspark
findspark.init()
#findspark.init("C:\\Users\\Vlad\\Documents\\SGH\\RTA\\spark")
#findspark.init("/Users/air/Desktop/spark/") # on my mac

### SparkContext

1. Główny, podstawowy obiekt
2. Punkt wejścia do pracy ze Sparkiem
3. Generowanie obiektów RDD

In [2]:
# inicjalizacja SparkContext
from pyspark import SparkContext
sc = SparkContext(appName="myAppName")

21/09/15 07:28:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
sc

<SparkContext master=local[*] appName=myAppName>

### SparkSession

1. Główny punkt wyjścia do SparkSQL
2. Opakowuje (wrapper) SparkContext
3. Zazwyczaj pierwszy obiekt, który będziemy tworzyć

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .appName("new")\
        .getOrCreate()
# otrzymanie obiektu SparkContext
sc = spark.sparkContext

In [5]:
spark

In [6]:
sc

<SparkContext master=local[*] appName=myAppName>

### RDD

- Resilient Distributed Dataset
- Podstawowa abstrakcja oraz rdzeń Sparka
- Obsługiwane przez dwa rodzaje operacji:
    - Akcje:
        - operacje uruchamiające/ egzekucję transformacji na RDD
        - przyjmują RDD jako input i zwracają wynik NIE będący RDD
    - Transformacje:
        - leniwe operacje
        - przyjmują RDD i zwracają RDD

- In-Memory - dane RDD przechowywane w pamięci
- Immutable 
- Lazy evaluated
- Parallel - przetwarzane równolegle
- Partitioned - rozproszone 

## WAŻNE informacje !

Ważne do zrozumienia działania SPARKA:

Term                   |Definition
----                   |-------
RDD                    |Resilient Distributed Dataset
Transformation         |Spark operation that produces an RDD
Action                 |Spark operation that produces a local object
Spark Job              |Sequence of transformations on data with a final action


Dwie podstawowe metody tworzenia RDD:

Method                      |Result
----------                               |-------
`sc.parallelize(array)`                  |Create RDD of elements of array (or list)
`sc.textFile(path/to/file)`                      |Create RDD of lines from file

Podstawowe transformacje

Transformation Example                          |Result
----------                               |-------
`filter(lambda x: x % 2 == 0)`           |Discard non-even elements
`map(lambda x: x * 2)`                   |Multiply each RDD element by `2`
`map(lambda x: x.split())`               |Split each string into words
`flatMap(lambda x: x.split())`           |Split each string into words and flatten sequence
`sample(withReplacement=True,0.25)`      |Create sample of 25% of elements with replacement
`union(rdd)`                             |Append `rdd` to existing RDD
`distinct()`                             |Remove duplicates in RDD
`sortBy(lambda x: x, ascending=False)`   |Sort elements in descending order

Podstawowe akcje 

Action                             |Result
----------                             |-------
`collect()`                            |Convert RDD to in-memory list 
`take(3)`                              |First 3 elements of RDD 
`top(3)`                               |Top 3 elements of RDD
`takeSample(withReplacement=True,3)`   |Create sample of 3 elements with replacement
`sum()`                                |Find element sum (assumes numeric elements)
`mean()`                               |Find element mean (assumes numeric elements)
`stdev()`                              |Find element deviation (assumes numeric elements)

-----
- **parallelize(c)** - tworzenie RDD na podstawie lokalnej kolekcji
- **map(f)** - zwraca nowe RDD po zastosowaniu podanej funkcji na każdym elemencie oryginalnego RDD (**T**)
- **filter(f)** - zwraca nowe RDD zawierające jedynie elementy które spełniają predykat (**T**)
- **reduce(f)** - agreguje elementy zbioru wykorzystując podaną funkcję. Funkcja redukująca musi być asocjacyjna [(a x b) x c = a x (b x c)] i przemienna [a x b = b x a] (**A**)

In [7]:
rdd = sc.parallelize(range(10)) # utworzenie RDD 

rdd

PythonRDD[1] at RDD at PythonRDD.scala:53

In [8]:
rdd.collect() # akcja 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [9]:
rdd.take(2)

[0, 1]

In [10]:
rdd.takeSample(True,3)

[5, 2, 7]

In [11]:
rdd.takeSample(False,3)

[8, 0, 4]

In [12]:
rdd.count()

10

In [13]:
rdd.mean()

4.5

In [14]:
%%file example.txt
first 
second line
the third line
then a fourth line

Overwriting example.txt


In [15]:
text_rdd = sc.textFile('example.txt')

In [16]:
text_rdd.first()

'first '

In [17]:
text_rdd.take(3)

['first ', 'second line', 'the third line']

In [18]:
text_rdd.takeSample(True,2)

['second line', 'the third line']

In [19]:
text_rdd.takeSample(False,2)

['first ', 'second line']

In [20]:
text_rdd.count()

4

In [21]:
rdd1 = sc.parallelize(range(1,20))
rdd2 = sc.parallelize(range(10,25))
rdd3 = rdd1.union(rdd2)
rdd3.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

In [22]:
rdd4 = rdd3.distinct()
rdd4.collect()

[8,
 16,
 24,
 1,
 9,
 17,
 2,
 10,
 18,
 3,
 11,
 19,
 4,
 12,
 20,
 5,
 13,
 21,
 6,
 14,
 22,
 7,
 15,
 23]

-----
- **parallelize(c)** - tworzenie RDD na podstawie lokalnej kolekcji
- **map(f)** - zwraca nowe RDD po zastosowaniu podanej funkcji na każdym elemencie oryginalnego RDD (**T**)
- **filter(f)** - zwraca nowe RDD zawierające jedynie elementy które spełniają predykat (**T**)
- **reduce(f)** - agreguje elementy zbioru wykorzystując podaną funkcję. Funkcja redukująca musi być asocjacyjna [(a x b) x c = a x (b x c)] i przemienna [a x b = b x a] (**A**)

#### Map vs. FlatMap

In [23]:
text_rdd.map(lambda line: line.split()).collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'fourth', 'line']]

In [24]:
# Collect everything as a single flat map
text_rdd.flatMap(lambda line: line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'fourth',
 'line']

In [25]:
sc.parallelize(range(20)) \
    .map(lambda x: x * 2) \
    .filter(lambda x: x != 2) \
    .reduce(lambda x,y: x + y)

378

In [26]:
sc.parallelize(range(20)).collect() # rozproszenie -> parallelize -> collect powrot

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [27]:
sc.parallelize(range(20)).map(lambda x: x * 2).collect()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38]

In [28]:
sc.parallelize(range(20)) \
    .map(lambda x: x * 2).filter(lambda x: x != 2).collect()

[0, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38]

----
- **textFile(p)** - tworzenie RDD na podstawie pliku. Jeden wiersz = jeden element RDD
- **flatMap(f)** - zwraca nowe RDD po zastosowaniu podanej funkcji na każdym elemencie oryginalnego RDD oraz spłaszczeniu rezultatu (**T**)
- **reduceByKey(f)** - zwraca RDD z połączonymi wartościami dla każdego klucza. Funkcja redukująca musi być asocjacyjna [(a x b) x c = a x (b x c)] i przemienna [a x b = b x a] (**T**)
- **collect()** - zwraca elementy RDD na driver (**A**)

In [29]:
import re

In [30]:
sc.textFile("RDD_input") \
    .map(lambda x: re.findall(r"[a-z']+", x.lower())) \
    .flatMap(lambda x: [(y, 1) for y in x]) \
    .reduceByKey(lambda x,y: x + y) \
    .collect()

[('is', 12),
 ('engine', 1),
 ('compatible', 1),
 ('hadoop', 3),
 ('run', 1),
 ('in', 3),
 ('clusters', 1),
 ('yarn', 1),
 ("spark's", 1),
 ('mode', 1),
 ('process', 1),
 ('hdfs', 1),
 ('cassandra', 1),
 ('hive', 1),
 ('designed', 1),
 ('perform', 1),
 ('both', 1),
 ('similar', 1),
 ('new', 1),
 ('like', 1),
 ('streaming', 1),
 ('machine', 1),
 ('learning', 1),
 ('than', 8),
 ('implicit', 1),
 ('simple', 1),
 ('complicated', 1),
 ('flat', 1),
 ('nested', 1),
 ('sparse', 1),
 ('readability', 1),
 ('counts', 1),
 ('cases', 1),
 ("aren't", 1),
 ('rules', 1),
 ('although', 3),
 ('errors', 1),
 ('never', 3),
 ('pass', 1),
 ('explicitly', 1),
 ('silenced', 1),
 ('of', 2),
 ('refuse', 1),
 ('temptation', 1),
 ('guess', 1),
 ('there', 1),
 ('only', 1),
 ('way', 2),
 ('do', 2),
 ('may', 2),
 ('at', 1),
 ("you're", 1),
 ('dutch', 1),
 ('now', 2),
 ('right', 1),
 ('idea', 3),
 ('good', 1),
 ('are', 1),
 ('honking', 1),
 ("let's", 1),
 ('more', 1),
 ('spark', 1),
 ('a', 3),
 ('fast', 1),
 ('and', 

In [31]:
sc.textFile("RDD_input") \
    .map(lambda x: re.findall(r"[a-z']+", x.lower())) \
    .flatMap(lambda x: [(y, 1) for y in x]).collect()

[('spark', 1),
 ('is', 1),
 ('a', 1),
 ('fast', 1),
 ('and', 1),
 ('general', 1),
 ('processing', 1),
 ('engine', 1),
 ('compatible', 1),
 ('with', 1),
 ('hadoop', 1),
 ('data', 1),
 ('it', 1),
 ('can', 1),
 ('run', 1),
 ('in', 1),
 ('hadoop', 1),
 ('clusters', 1),
 ('through', 1),
 ('yarn', 1),
 ('or', 1),
 ("spark's", 1),
 ('standalone', 1),
 ('mode', 1),
 ('and', 1),
 ('it', 1),
 ('can', 1),
 ('process', 1),
 ('data', 1),
 ('in', 1),
 ('hdfs', 1),
 ('hbase', 1),
 ('cassandra', 1),
 ('hive', 1),
 ('and', 1),
 ('any', 1),
 ('hadoop', 1),
 ('inputformat', 1),
 ('it', 1),
 ('is', 1),
 ('designed', 1),
 ('to', 1),
 ('perform', 1),
 ('both', 1),
 ('batch', 1),
 ('processing', 1),
 ('similar', 1),
 ('to', 1),
 ('mapreduce', 1),
 ('and', 1),
 ('new', 1),
 ('workloads', 1),
 ('like', 1),
 ('streaming', 1),
 ('interactive', 1),
 ('queries', 1),
 ('and', 1),
 ('machine', 1),
 ('learning', 1),
 ('beautiful', 1),
 ('is', 1),
 ('better', 1),
 ('than', 1),
 ('ugly', 1),
 ('explicit', 1),
 ('is', 1

In [32]:
sc.textFile("RDD_input") \
    .map(lambda x: re.findall(r"[a-z']+", x.lower())).collect()

[['spark',
  'is',
  'a',
  'fast',
  'and',
  'general',
  'processing',
  'engine',
  'compatible',
  'with',
  'hadoop',
  'data'],
 ['it',
  'can',
  'run',
  'in',
  'hadoop',
  'clusters',
  'through',
  'yarn',
  'or',
  "spark's",
  'standalone',
  'mode'],
 ['and',
  'it',
  'can',
  'process',
  'data',
  'in',
  'hdfs',
  'hbase',
  'cassandra',
  'hive',
  'and',
  'any',
  'hadoop',
  'inputformat'],
 ['it',
  'is',
  'designed',
  'to',
  'perform',
  'both',
  'batch',
  'processing',
  'similar',
  'to',
  'mapreduce'],
 ['and',
  'new',
  'workloads',
  'like',
  'streaming',
  'interactive',
  'queries',
  'and',
  'machine',
  'learning'],
 [],
 [],
 ['beautiful', 'is', 'better', 'than', 'ugly'],
 ['explicit', 'is', 'better', 'than', 'implicit'],
 ['simple', 'is', 'better', 'than', 'complex'],
 ['complex', 'is', 'better', 'than', 'complicated'],
 ['flat', 'is', 'better', 'than', 'nested'],
 ['sparse', 'is', 'better', 'than', 'dense'],
 ['readability', 'counts'],
 ['s

> ZADANIE

In [33]:
rawMD = sc.textFile("MobyDick.txt")
rawMD.take(10)

['MOBY DICK;',
 '',
 '',
 'or, THE WHALE.',
 '',
 '',
 'CHAPTER 1. Loomings.',
 '',
 'Call me Ishmael. Some years ago—never mind how long precisely—having',
 'little or no money in my purse, and nothing particular to interest me on']

### SQL 

In [35]:
adultDF = spark.read.csv("adult.data", inferSchema=True, ignoreLeadingWhiteSpace=True)

In [36]:
adultDF.take(1)

[Row(_c0=39, _c1='State-gov', _c2=77516, _c3='Bachelors', _c4=13, _c5='Never-married', _c6='Adm-clerical', _c7='Not-in-family', _c8='White', _c9='Male', _c10=2174, _c11=0, _c12=40, _c13='United-States', _c14='<=50K')]

In [37]:
col_names = ["age", "workclass", "fnlwgt", "education", "education-num","marital-status", "occupation", 
             "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", 
             "native-country", "earnings"]

In [38]:
adultDF = adultDF.toDF(*col_names).drop("fnlwgt").dropna("any")

In [39]:
adultDF.show(3, vertical=True)

-RECORD 0----------------------------
 age            | 39                 
 workclass      | State-gov          
 education      | Bachelors          
 education-num  | 13                 
 marital-status | Never-married      
 occupation     | Adm-clerical       
 relationship   | Not-in-family      
 race           | White              
 sex            | Male               
 capital-gain   | 2174               
 capital-loss   | 0                  
 hours-per-week | 40                 
 native-country | United-States      
 earnings       | <=50K              
-RECORD 1----------------------------
 age            | 50                 
 workclass      | Self-emp-not-inc   
 education      | Bachelors          
 education-num  | 13                 
 marital-status | Married-civ-spouse 
 occupation     | Exec-managerial    
 relationship   | Husband            
 race           | White              
 sex            | Male               
 capital-gain   | 0                  
 capital-los

In [40]:
adultDF.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: string (nullable = true)
 |-- earnings: string (nullable = true)



In [41]:
adultDF.show(3)

+---+----------------+---------+-------------+------------------+-----------------+-------------+-----+----+------------+------------+--------------+--------------+--------+
|age|       workclass|education|education-num|    marital-status|       occupation| relationship| race| sex|capital-gain|capital-loss|hours-per-week|native-country|earnings|
+---+----------------+---------+-------------+------------------+-----------------+-------------+-----+----+------------+------------+--------------+--------------+--------+
| 39|       State-gov|Bachelors|           13|     Never-married|     Adm-clerical|Not-in-family|White|Male|        2174|           0|            40| United-States|   <=50K|
| 50|Self-emp-not-inc|Bachelors|           13|Married-civ-spouse|  Exec-managerial|      Husband|White|Male|           0|           0|            13| United-States|   <=50K|
| 38|         Private|  HS-grad|            9|          Divorced|Handlers-cleaners|Not-in-family|White|Male|           0|         

In [42]:
adultDF.groupBy('education').mean().show()

+------------+------------------+------------------+------------------+------------------+-------------------+
|   education|          avg(age)|avg(education-num)| avg(capital-gain)| avg(capital-loss)|avg(hours-per-week)|
+------------+------------------+------------------+------------------+------------------+-------------------+
|     Masters| 44.04991294254208|              14.0| 2562.563551944283|166.71967498549043|  43.83633197910621|
|        10th| 37.42979635584137|               6.0|404.57449088960345|56.845659163987136| 37.052518756698824|
|     5th-6th|42.885885885885884|               3.0|176.02102102102103| 68.25225225225225|   38.8978978978979|
|  Assoc-acdm|37.381443298969074|              12.0| 640.3992502343018| 93.41893158388004| 40.504217432052485|
|   Assoc-voc| 38.55354558610709|              11.0| 715.0513748191028| 72.75470332850941|  41.61070911722142|
|     7th-8th| 48.44582043343653|               4.0|233.93962848297213|  65.6687306501548|  39.36687306501548|
|

In [43]:
adultDF.write.saveAsTable("adult_bucket")

In [44]:
newAdult = spark.sql("select age, education, sex from adult_bucket where age > 50")

In [45]:
newAdult.show(3)

+---+---------+------+
|age|education|   sex|
+---+---------+------+
| 53|     11th|  Male|
| 52|  HS-grad|  Male|
| 54|  HS-grad|Female|
+---+---------+------+
only showing top 3 rows

